In [13]:
from datasets import load_dataset
import pandas as pd
import json

# Load the dataset from Hugging Face
dataset = load_dataset('TIGER-Lab/MathInstruct')  # Replace 'dataset_name' with the name of the dataset you want to load

# Convert to a Pandas DataFrame
df = pd.DataFrame(dataset['train'])  # Use 'train', 'test', or 'validation' based on your needs
df

Found cached dataset json (/Users/bryan/.cache/huggingface/datasets/TIGER-Lab___json/TIGER-Lab--MathInstruct-5c606c36c8364f7c/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

,instruction,output,source
0,The distance between two stars is 6.52 × 10^5 light years. What is the distance between the two stars in parsecs? (1 parsec = 3.26 light years)\nAnswer Choices: (A) 2 × 10^5 (B) 4 × 10^6 (C) 5 × 10^7 (D) 7 × 10^7 (E) 9 × 10^8,Let's think about the multi-choice question.\n6.52 × 10^5 ly / (3.26 ly/parsec) = 2 x 10^5 persec\nThe answer is A.,data/CoT/aqua_rat.json
1,How many ways can the letters in the word COMMON be arranged?\nAnswer Choices: (A) 6 (B) 30 (C) 90 (D) 120 (E) 180,Let's solve the multi-choice question step by step.\nAccording to the above the # of permutations of 6 letters COMMON out of which 2 O's and 2 M's are identical is 6!2!∗2!=180\nThe answer is E.,data/CoT/aqua_rat.json
2,"A team of six entered for a shooting competition. The best marks man scored 85 points. If he had scored 92 points, the average scores for. The team would have been 84. How many points altogether did the team score?\nAnswer Choices: (A) 288 (B) 497 (C) 168 (D) 127 (E) 664 Let's program in Python in the response.","answers = ['A', 'B', 'C', 'D', 'E']\n# If the best marksman had scored 92 points, the total score would have been 84 * 6 = 504\n# But he actually scored 85 points, so the actual total score is 504 - 92 + 85\nactual_total_score = 504 - 92 + 85\noptions = [288, 497, 168, 127, 664]\nindex = options.index(actual_total_score)\nprint(answers[index])",data/PoT/aqua_rat_filtered.json
3,A psychiatrist has 4 patients that need 25 sessions in total. One of the patients needs 6 sessions. Another patient needs 5 more than that. How many sessions would the remaining patients need?,The second patient needs 6+5 = 11 sessions\n25-11-6 = 8 sessions\nThe answer is 8,data/CoT/gsm_rft.json
4,The radius of a wheel is 22.4 cm. What is the distance covered by the wheel in making 500 resolutions?\nAnswer Choices: (A) 187 m (B) 704 m (C) 179 m (D) 127 m (E) 297 m Let's write a Python program.,radius = 22.4\nresolutions = 500\n# calculate the circumference of the wheel\ncircumference = 2 * 3.14 * radius\n# calculate the distance covered by the wheel in making 500 resolutions\ndistance = circumference * resolutions\nprint(distance),data/PoT/aqua_rat_filtered.json
...,...,...,...
262780,"There are 81 people in a certain community who belong to the local golf club, tennis club or both clubs. If twice as many people belong to the tennis club as belong to the golf club, what could be the number of people who belong to both clubs?\nAnswer Choices: (A) 4 (B) 7 (C) 27 (D) 21 (E) 37",Let's think about the multi-choice question.\nlet's find the range of Both and hence the range of x. Minimum value of Both: Say there is no overlap in the two sets. 81 = x + 2x\rx = 27\rIn this case Both = 0 Maximum value of Both: Say there is maximum overlap between the two sets. One set has x so it could be a subset of the set 2x.\r81 = 2x\rBut x cannot be a decimal so (x-1) could be the overlap (= Both) and there could be 1 in only the Golf club.\r81 = 2 x + x - (x - 1)\rx = 40\rIn this case Both = 40 - 1 = 39 So Both lies between 0 and 39 Only option E satisfies this condition. The answer is E,data/CoT/aqua_rat.json
262781,"How many 3 digit numbers can be made from the numbers 1,2,3,4,5,6,7,8,9,0\nAnswer Choices: (A) 1000 (B) 720 (C) 648 (D) 500 (E) 100","Let's solve the multi-choice question step by step.\nA3 digit number is made from (First Digit , Second digit, Third digit) ABC\nFor the first digit of the 3 digit numbers we have to choose from numbers 1,2,3,4,5,6,7,8,9 we cannot take 0 since it is a 3 digit number that we have to make\nSo the first digit can have 9 entries.\nFor the second digit we can have all numbers including 0 except one number which we used for first digit. So we have 9 choices (including 0)\nFor the third digit we have all numbers minus two digits which we used for A and B\nSo we have 8 choices.\nTotal ways by which we can form a 3 digit number are 9 * 9 * 8 = 648\nThe answer is C",data/CoT/aqua_rat.json
262782,"The sum of the 

In [14]:
df = df[df['source'].str.contains('PoT')]
df

,instruction,output,source
2,"A team of six entered for a shooting competition. The best marks man scored 85 points. If he had scored 92 points, the average scores for. The team would have been 84. How many points altogether did the team score?\nAnswer Choices: (A) 288 (B) 497 (C) 168 (D) 127 (E) 664 Let's program in Python in the response.","answers = ['A', 'B', 'C', 'D', 'E']\n# If the best marksman had scored 92 points, the total score would have been 84 * 6 = 504\n# But he actually scored 85 points, so the actual total score is 504 - 92 + 85\nactual_total_score = 504 - 92 + 85\noptions = [288, 497, 168, 127, 664]\nindex = options.index(actual_total_score)\nprint(answers[index])",data/PoT/aqua_rat_filtered.json
4,The radius of a wheel is 22.4 cm. What is the distance covered by the wheel in making 500 resolutions?\nAnswer Choices: (A) 187 m (B) 704 m (C) 179 m (D) 127 m (E) 297 m Let's write a Python program.,radius = 22.4\nresolutions = 500\n# calculate the circumference of the wheel\ncircumference = 2 * 3.14 * radius\n# calculate the distance covered by the wheel in making 500 resolutions\ndistance = circumference * resolutions\nprint(distance),data/PoT/aqua_rat_filtered.json
6,"At a pool party, there are 4 pizzas cut into 12 slices each. If the guests eat 39 slices, how many slices are left? Let's write a Python program.",# define the initial number of slices\ntotal_slices = 4 * 12\n# define the number of slices eaten\neaten_slices = 39\n# calculate the number of slices left\nleft_slices = total_slices - eaten_slices\n# print the result\nprint(left_slices),data/PoT/gsm_gpt4.json
7,"Noel bakes 4 dozen donuts for his class. There are 30 students in class, but only 80% like donuts. How many donuts does each student who likes donuts get to eat? Please write a program to solve it",# define the variables\ntotal_donuts = 4 * 12 # since a dozen is 12\ntotal_students = 30\npercentage_like_donuts = 0.8 # 80%\n\n# calculate the number of students who like donuts\nstudents_like_donuts = total_students * percentage_like_donuts\n\n# calculate the number of donuts each student who likes donuts gets\ndonuts_per_student = total_donuts / students_like_donuts\n\n# print the result\nprint(donuts_per_student),data/PoT/gsm_gpt4.json
9,"louie takes out a 3 - month loan of $ 1000 . the lender charges him 10 % interest per month compounded monthly . the terms of the loan state that louie must repay the loan in 3 equal monthly payments . to the nearest dollar , how much does louis have to pay each month ? Let's write a program.",n0 = 3.0\nn1 = 1000.0\nn2 = 10.0\nn3 = 3.0\nt0 = n2 / 100.0\nt1 = n0 * n1\nt2 = t0 * t1\nt3 = t0 * t2\nt4 = t2 + t3\nt5 = t4 + 1.0\nt6 = n1 + t5\nt7 = t5 / 100.0\nanswer = t6 / t7\nprint(answer),data/PoT/mathqa.json
...,...,...,...
262754,"Four of the six numbers 1867, 1993, 2019, 2025, 2109, and 2121 have a mean (average) of 2008. What is the mean (average) of the other two numbers? Please respond by writing a program in Python.","# define the list of numbers\nnumbers = [1867, 1993, 2019, 2025, 2109, 2121]\n\n# calculate the total sum of all numbers\ntotal_sum = sum(numbers)\n\n# calculate the sum of the four numbers that have a mean of 2008\nsum_of_four = 4 * 2008\n\n# calculate the sum of the other two numbers\nsum_of_other_two = total_sum - sum_of_four\n\n# calculate the mean of the other two numbers\nmean_of_other_two = sum_of_other_two / 2\n\n# print the result\nprint(mean_of_other_two)",data/PoT/MATH_train.json
262767,a vendor sells 60 percent of apples he had and throws away 15 percent of the remainder . next day he sells 50 percent of the remainder and throws away the rest . what percent of his apples does the vendor throw ? Let's write a Python program to solve it.,n0 = 60.0\nn1 = 15.0\nn2 = 50.0\nt0 = n1 / 100.0\nt1 = n2 / 100.0\nt2 = 100.0 - n0\nt3 = t0 * t2\nt4 = t2 - t3\nt5 = t1 * t4\nanswer = t3 + t5\nprint(answer),data/PoT/mathqa.json
262771,"let p be a prime number . if n = 7 p , how many divisors does n 

In [15]:
df['output2'] = "Question: " + df['instruction'] + "\n" + "Solution:\n" + df['output']
pd.set_option('display.max_colwidth', -1)
df

<ipython-input-15-829171d7a50c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['output2'] = "Question: " + df['instruction'] + "/n" + "Solution:/n" + df['output']
<ipython-input-15-829171d7a50c>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,instruction,output,source,output2
2,"A team of six entered for a shooting competition. The best marks man scored 85 points. If he had scored 92 points, the average scores for. The team would have been 84. How many points altogether did the team score?\nAnswer Choices: (A) 288 (B) 497 (C) 168 (D) 127 (E) 664 Let's program in Python in the response.","answers = ['A', 'B', 'C', 'D', 'E']\n# If the best marksman had scored 92 points, the total score would have been 84 * 6 = 504\n# But he actually scored 85 points, so the actual total score is 504 - 92 + 85\nactual_total_score = 504 - 92 + 85\noptions = [288, 497, 168, 127, 664]\nindex = options.index(actual_total_score)\nprint(answers[index])",data/PoT/aqua_rat_filtered.json,"Question: A team of six entered for a shooting competition. The best marks man scored 85 points. If he had scored 92 points, the average scores for. The team would have been 84. How many points altogether did the team score?\nAnswer Choices: (A) 288 (B) 497 (C) 168 (D) 127 (E) 664 Let's program in Python in the response./nSolution:/nanswers = ['A', 'B', 'C', 'D', 'E']\n# If the best marksman had scored 92 points, the total score would have been 84 * 6 = 504\n# But he actually scored 85 points, so the actual total score is 504 - 92 + 85\nactual_total_score = 504 - 92 + 85\noptions = [288, 497, 168, 127, 664]\nindex = options.index(actual_total_score)\nprint(answers[index])"
4,The radius of a wheel is 22.4 cm. What is the distance covered by the wheel in making 500 resolutions?\nAnswer Choices: (A) 187 m (B) 704 m (C) 179 m (D) 127 m (E) 297 m Let's write a Python program.,radius = 22.4\nresolutions = 500\n# calculate the circumference of the wheel\ncircumference = 2 * 3.14 * radius\n# calculate the distance covered by the wheel in making 500 resolutions\ndistance = circumference * resolutions\nprint(distance),data/PoT/aqua_rat_filtered.json,Question: The radius of a wheel is 22.4 cm. What is the distance covered by the wheel in making 500 resolutions?\nAnswer Choices: (A) 187 m (B) 704 m (C) 179 m (D) 127 m (E) 297 m Let's write a Python program./nSolution:/nradius = 22.4\nresolutions = 500\n# calculate the circumference of the wheel\ncircumference = 2 * 3.14 * radius\n# calculate the distance covered by the wheel in making 500 resolutions\ndistance = circumference * resolutions\nprint(distance)
6,"At a pool party, there are 4 pizzas cut into 12 slices each. If the guests eat 39 slices, how many slices are left? Let's write a Python program.",# define the initial number of slices\ntotal_slices = 4 * 12\n# define the number of slices eaten\neaten_slices = 39\n# calculate the number of slices left\nleft_slices = total_slices - eaten_slices\n# print the result\nprint(left_slices),data/PoT/gsm_gpt4.json,"Question: At a pool party, there are 4 pizzas cut into 12 slices each. If the guests eat 39 slices, how many slices are left? Let's write a Python program./nSolution:/n# define the initial number of slices\ntotal_slices = 4 * 12\n# define the number of slices eaten\neaten_slices = 39\n# calculate the number of slices left\nleft_slices = total_slices - eaten_slices\n# print the result\nprint(left_slices)"
7,"Noel bakes 4 dozen donuts for his class. There are 30 students in class, but only 80% like donuts. How many donuts does each student who likes donuts get to eat? Please write a program to solve it",# define the variables\ntotal_donuts = 4 * 12 # since a dozen is 12\ntotal_students = 30\npercentage_like_donuts = 0.8 # 80%\n\n# calculate the number of students who like donuts\nstudents_like_donuts = total_students * percentage_like_donuts\n\n# calculate the number of donuts each student who likes donuts gets\ndonuts_per_student = total_donuts / students_like_donuts\n\n# print the result\nprint(donuts_per_student),data/PoT/gsm_gpt4.json,"Question: Noel bakes 4 dozen donuts for his class. There are 30 students in class, but only 80% like donuts. How many donuts does each student who likes donuts get to eat? Please

In [ ]:
# Save DataFrame to a JSON file
data.to_json('dataset.json', orient='records', lines=True)  # Saves each row as a separate JSON object in the file